In [ ]:
import json, textwrap

# Collect credentials

In [ ]:
import os
from getpass import getpass

In [ ]:
# Check if running in Google Colab
try:
    from google.colab import userdata
    IS_COLAB = True
    print("Running in Google Colab environment")
except ImportError:
    IS_COLAB = False
    print("Running in local environment")

# Initialize variables
OPENAI_KEY = None
OPENAI_SECRET = None
credentials_entered_manually = False

# Method 1: Try loading from .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_KEY = os.getenv('OPENAI_KEY')
    OPENAI_SECRET = os.getenv('OPENAI_SECRET')
    if OPENAI_KEY and OPENAI_SECRET:
        print("✓ Credentials loaded from .env file")
    else:
        raise ValueError("Credentials not found in .env file")
except (ImportError, ValueError):
    # Method 2: Try Google Colab secrets (only if in Colab)
    if IS_COLAB:
        try:
            OPENAI_KEY = userdata.get('OPENAI_KEY')
            OPENAI_SECRET = userdata.get('OPENAI_SECRET')
            print("✓ Credentials loaded from Google Colab secrets")
        except Exception:
            # Method 3: Get credentials via user input
            print("Credentials not found in Colab secrets.")
    else:
        # Method 3: Get credentials via user input (local environment)
        print("Credentials not found in .env file.")
    if not OPENAI_KEY and not OPENAI_SECRET:
      print("Please enter your OpenAI credentials:")
      OPENAI_KEY = getpass("Enter your OpenAI API Key: ")
      OPENAI_SECRET = getpass("Enter your OpenAI Secret: ")
      credentials_entered_manually = True
      print("✓ Credentials entered manually")

# Verify credentials were obtained
if not OPENAI_KEY or not OPENAI_SECRET:
    raise ValueError("Failed to obtain OpenAI credentials")

# Save credentials if they were entered manually
if credentials_entered_manually:
    if IS_COLAB:
        print("\n" + "="*60)
        print("TO SAVE CREDENTIALS FOR FUTURE USE:")
        print("1. Click the 🔑 key icon in the left sidebar")
        print("2. Add a new secret with name: OPENAI_KEY")
        print("3. Add a new secret with name: OPENAI_SECRET")
        print("4. Paste your respective values and enable 'Notebook access'")
        print("="*60)
    else:
        # Save to .env file
        try:
            with open('.env', 'a') as f:
                # Check if file is empty or doesn't end with newline
                f.seek(0, 2)  # Go to end of file
                if f.tell() > 0:  # File is not empty
                    f.seek(-1, 2)  # Go to last character
                    if f.read(1) != '\n':
                        f.write('\n')

                f.write(f"OPENAI_KEY={OPENAI_KEY}\n")
                f.write(f"OPENAI_SECRET={OPENAI_SECRET}\n")
            print("✓ Credentials saved to .env file for future use")
        except Exception as e:
            print(f"⚠ Could not save to .env file: {e}")

print("Credentials successfully loaded and ready to use!")

In [ ]:
harvard_api = 'https://go.apis.huit.harvard.edu/ais-openai-direct-limited-schools/v1'

In [ ]:
from openai import OpenAI, BaseModel
client = OpenAI(
    api_key=OPENAI_KEY,
    base_url=harvard_api
)

In [ ]:
raw_text = """
Of Man’s first disobedience, and the fruit
Of that forbidden tree whose mortal taste
Brought death into the World, and all our woe,
With loss of Eden, till one greater Man
Restore us, and regain the blissful seat,
Sing Heavenly Muse, that, on the secret top
Of Oreb or of Sinai, didst inspire
That shepherd who first taught the chosen seed,
In the beginning how the heavens and earth
Rose out of Chaos: or if Sion Hill
Delight thee more, and Siloa’s brook that flow’d
Fast by the oracle of God; I thence
Invoke thy aid to my advent’rous song,
That with no middle flight intends to soar
Above th’ Aonian mount, while it pursues
Things unattempted yet in prose or rhyme.
"""

class PLExcerpt(BaseModel):
    book: int
    lines: str
    characters: list[str]
    themes: list[str]
    summary: str

    model_config = {"extra": "forbid"}

schema = PLExcerpt.model_json_schema()

In [ ]:
response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "Extract structured data about the passage. Make sure the summary is concise and technical, fitting for an academic well-versed in the text. Remove any filler or aggrandizement."
                       "Return JSON only—no extra keys or text."
        },
        {
            "role": "user",
            "content": raw_text
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "paradise_lost_excerpt",
            "strict": True,
            "schema": schema,
        }
    },
)

print("Parsed object:\n")

parsed = json.loads(response.output_text)
parsed["summary"] = textwrap.fill(parsed["summary"], width=80)
print(json.dumps(parsed, indent=2))

In [ ]:
import requests

In [ ]:
client.api_base = harvard_api

In [ ]:
url = "https://go.apis.huit.harvard.edu/ais-openai-direct-limited-schools/v1/chat/completions"
payload = json.dumps({
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "user",
      "content": "To what extent does scientific conservatism limit new discoveries?"
    }
  ],
  "temperature": 0.7
})
headers = {
  'Content-Type': 'application/json',
  'api-key': OPENAI_KEY
}
response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)

In [ ]:
response.json()